<a href="https://colab.research.google.com/github/zuzannazak/SentimentAnalysisInPolish/blob/main/Sentiment_analysis_in_Polish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Combining multiple files
**Input:** Csv files relating to 1 profile gathered from https://colab.research.google.com/drive/1P0mf1jhhYLqlDVN2eqC2HjbWeEg-6dmJ#scrollTo=QhTw_07h092i, uploaded to Colab

**Output:** One CSV file

In [14]:
import os
import glob
import pandas as pd
os.chdir("/content")

In [3]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [4]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

#Sentiment analysis
**Input:** Prepared csv files gathered from https://colab.research.google.com/drive/1P0mf1jhhYLqlDVN2eqC2HjbWeEg-6dmJ#scrollTo=QhTw_07h092i and combined per social media account

**Output:** sentiment assigned to each tweet

In [5]:
pip install -i https://pypi.clarin-pl.eu/ lpmn_client

Looking in indexes: https://pypi.clarin-pl.eu/


In [6]:
 pip show lpmn_client

Name: lpmn-client
Version: 1.4.3
Summary: Client for writing LPMN queries.
Home-page: UNKNOWN
Author: Szymon Ciombor, Mateusz Gniewkowski
Author-email: Szymon.Ciombor@pwr.edu.pl, mateusz.gniewkowski@pwr.edu.pl
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: pyyaml, tqdm, requests
Required-by: 


#Sentiment analysis from csv file


In [8]:
import pandas as pd
import numpy as np

import os

from lpmn_client import download_file, upload_file
from lpmn_client import Task

In [15]:
df = pd.read_csv("/content/SearchTweetPaczkomatyPL('2022-01-22T07_05_13.009Z',).csv")
df['sentiment'] = None
df = df.drop(columns=['Unnamed: 0'])

In [10]:
df.head()

,text,in_reply_to_user_id,id,lang,author_id,referenced_tweets,reply_settings,public_metrics,created_at,source,conversation_id,geo,sentiment
0,@K_Kielczewski @PaczkomatyPL przyznacie się sa...,3.296933e+09,1482855036799623171,pl,819868801936293889,"[{'type': 'replied_to', 'id': '148278301475327...",everyone,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",2022-01-16T23:19:25.000Z,Twitter for iPhone,1482783014753271808,NaN,None
1,@r6_reggie @sg4rr_ @PaczkomatyPL XDD,1.385887e+18,1482710846056841224,und,1378155874913509377,"[{'type': 'replied_to', 'id': '148270418562800...",everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-01-16T13:46:27.000Z,Twitter for Android,1482630913137778692,NaN,None
2,@Damian43211 @sg4rr_ @PaczkomatyPL Temu co pac...,1.378156e+18,1482704185628008451,pl,1385886999769403392,"[{'type': 'replied_to', 'id': '148267913136826...",everyone,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",2022-01-16T13:19:59.000Z,Twitter Web App,1482630913137778692,NaN,None
3,@PaczkomatyPL A tam bot jest \nJezus Maria 🖕,2.565559e+09,1482682057323077638,pl,3173102986,"[{'type': 'replied_to', 'id': '148268088370635...",everyone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-01-16T11:52:04.000Z,Twitter for Android,1482330351938805760,NaN,None
4,@PaczkomatyPL @PaczkomatyPL \nI co...\nTeraz.....,2.565559e+09,1482680022594957315,pl,3173102986,"[{'type': 'replied_to', 'id': '148234984800752...",everyone,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",2022-01-16T11:43:58.000Z,Twitter for Android,1482330351938805760,NaN,None


In [ ]:
#szczegóły analizy sentymentu:
#    Za ujednoznacznienie odpowiedzialne jest narzędzie "wsd" i ono niestety może popełniać drobne błędy. 
#    W narzędziu ccl_emo ostateczny wydźwięk jest brany nie z pojedynczego pojęcia a z całego synsetu (synonimy) poprzez głosowanie większościowe.

for i, row in df.iterrows():
    unique_id = i
    val=(df['text'].values[i])
    with open('plik.txt', 'w') as f:
      f.write(val)
    file_id = upload_file("/content/plik.txt")
    t = Task('any2txt|wcrft2|wsd|ccl_emo({"lang":"polish"})|ccl_emo_stats({"lang":"polish", "split_paragraphs": false}) ')
    output_file_id = t.run(file_id)
    downloaded = download_file(output_file_id, "./out")

    try:
      df_sentiment=pd.read_csv(downloaded, compression='zip', header=0, sep=';')
      df_sentiment = df_sentiment.replace("None", 0)
      df_sentiment = df_sentiment.replace("amb", 0)
      df_sentiment["Polarity"] = pd.to_numeric(df_sentiment["Polarity"])
      column_sum = df_sentiment["Polarity"].sum()
      df['sentiment'].values[i]= column_sum
      print(df['text'].values[i] + ": " + str(column_sum))
    except:
      df['sentiment'].values[i]= None
      print(df['text'].values[i] + ": no sentiment for value " + str(i) )

df.to_csv('Paczkomatysentyment.csv')


@K_Kielczewski @PaczkomatyPL przyznacie się sami ze wasi kierowcy jeżdżą po chodniku i ze stawiacie paczkomaty w takich lokalizacjach gdzie nie da się ich normalnie obsługiwać czy jak zwykle powiecie ze wszystko ok i ze zawsze pęknięcie przepisowo? 🤡: -1
@r6_reggie @sg4rr_ @PaczkomatyPL XDD: no sentiment for value 1
@Damian43211 @sg4rr_ @PaczkomatyPL Temu co paczkomat odpisał xD: 0
@PaczkomatyPL A tam bot jest 
Jezus Maria 🖕: 0
@PaczkomatyPL @PaczkomatyPL 
I co...
Teraz...
Beton botów czy jakiś człowiek ze mną pogada https://t.co/1lbHSpjZUr: 0
@sg4rr_ @PaczkomatyPL Ten co kurwa paczkomat wyjaśnia 🤣🤣🤣🤣: -1
@sg4rr_ @PaczkomatyPL XDDDD: no sentiment for value 6
@PaczkomatyPL cześć! Zastanawia mnie czy jak minie 48 godzin to skrytka jest zdalnie blokowana przed otwarciem czy dopóki kurier nie wyciągnie paczki to mogę odebrać?: 0
@Hirosu34583422 @PaczkomatyPL chłopie co ty w ogole pierdzielisz, ja czekam na paczkę OD LISTOPADA, miała być niby przed świętami, a dalej jej nie mam: 0
KURWA MAĆ

In [ ]:
#szczegóły analizy sentymentu:
#    Za ujednoznacznienie odpowiedzialne jest narzędzie "wsd" i ono niestety może popełniać drobne błędy - zostało pominiete
#    W narzędziu ccl_emo ostateczny wydźwięk jest brany nie z pojedynczego pojęcia a z całego synsetu (synonimy) poprzez głosowanie większościowe.

for i, row in df.iterrows():
    unique_id = i
    val=(df['text'].values[i])
    with open('plik.txt', 'w') as f:
      f.write(val)
    file_id = upload_file("/content/plik.txt")
    t = Task('any2txt|wcrft2|ccl_emo({"lang":"polish"})|ccl_emo_stats({"lang":"polish", "split_paragraphs": false}) ')
    output_file_id = t.run(file_id)
    downloaded = download_file(output_file_id, "./out")

    try:
      df_sentiment=pd.read_csv(downloaded, compression='zip', header=0, sep=';')
      df_sentiment = df_sentiment.replace("None", 0)
      df_sentiment = df_sentiment.replace("amb", 0)
      df_sentiment["Polarity"] = pd.to_numeric(df_sentiment["Polarity"])
      column_sum = df_sentiment["Polarity"].sum()
      df['sentiment'].values[i]= column_sum
      print(df['text'].values[i] + ": " + str(column_sum))
    except:
      df['sentiment'].values[i]= None
      print(df['text'].values[i] + ": no sentiment for value " + str(i) )

df.to_csv('Paczkomatysentyment.csv')


##Sentiment check per zdanie

In [ ]:
sentiment_to_check = "@PaczkomatyPL Nie dostarczył do oddziału, trzyma je na samochodzie. Do świąt nie dojdą do odbiorcy na 95%"

In [ ]:
val=(sentiment_to_check)
with open('plik.txt', 'w') as f:
  f.write(val)
file_id = upload_file("/content/plik.txt")
t = Task('any2txt|wcrft2|wsd|ccl_emo({"lang":"polish"})|ccl_emo_stats({"lang":"polish", "split_paragraphs": false}) ')
output_file_id = t.run(file_id)
downloaded = download_file(output_file_id, "./out")

try:
    check_sentiment=pd.read_csv(downloaded, compression='zip', header=0, sep=';')
    check_sentiment = check_sentiment.replace("None", 0)
    check_sentiment = check_sentiment.replace("amb", 0)
    check_sentiment["Polarity"] = pd.to_numeric(check_sentiment["Polarity"])
    column_sum = check_sentiment["Polarity"].sum()
    print(sentiment_to_check + ": " + str(column_sum))
except:
    print(sentiment_to_check + ": no sentiment!")


In [ ]:
df_sentiment

,Lemma,Syn_id,Polarity,Emotion_names,Emotion_valuations,Units in synset
0,korzystać,1377,0,NaN,NaN,czerpać_korzyści.1(38:cpos) dobrze_wychodzić.1...
1,konkurencja,1340,0,NaN,NaN,kompetycja.1(8:zdarz) wyścig.2(8:zdarz) współz...


In [ ]:
df

,source,conversation_id,text,created_at,lang,in_reply_to_user_id,reply_settings,author_id,id,referenced_tweets,public_metrics,geo,sentiment
0,Twitter for Android,1475050133461032963,"@PaczkomatyPL Pewnie, że tak 😊 Już wszystko za...",2021-12-26T10:40:33.000Z,pl,2.565559e+09,everyone,52522222,1475053917746696196,"[{'type': 'replied_to', 'id': '147505300392113...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,-2
1,Twitter for Android,1475050133461032963,"Pierwszy raz mam taką sytuację, że nie mogę od...",2021-12-26T10:25:31.000Z,pl,NaN,everyone,52522222,1475050133461032963,NaN,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,None
2,Twitter Web App,1474893391817232390,@PaczkomatyPL \nJest jakiś problem po waszej s...,2021-12-26T00:02:41.000Z,pl,2.565559e+09,everyone,2799071567,1474893391817232390,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
3,Twitter for Android,1474623367227617285,@CicheMale @MeteoprognozaPL @ebonit_7 @Doppelm...,2021-12-25T22:04:57.000Z,pl,7.113024e+17,everyone,1376873115091881986,1474863762792493059,"[{'type': 'replied_to', 'id': '147462336722761...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
4,Twitter Web App,1474623367227617285,@CicheMale @ebonit_7 @DoppelmayrGmbH @_PistenB...,2021-12-25T11:42:00.000Z,pl,7.113024e+17,everyone,2651114600,1474706994225463301,"[{'type': 'replied_to', 'id': '147462336722761...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,Twitter for Android,1479235375117778946,@OLogistyczny @InPostPL1 Chyba znowu oznaczyli...,2022-01-06T23:43:47.000Z,pl,1.382288e+18,everyone,1227343711278944257,1479237288097943555,"[{'type': 'replied_to', 'id': '147923537511777...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
937,Twitter for Android,1478035206904950788,@PaczkomatyPL @RBrzoska Czyli mam czekać 30 dn...,2022-01-06T18:23:59.000Z,pl,8.590470e+17,everyone,859047002381967361,1479156810233700355,"[{'type': 'replied_to', 'id': '147908802222645...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
938,Twitter for iPad,1478833901510475785,@PaczkomatyPL @HERYTlERA A czy w piątek kurier...,2022-01-06T16:24:41.000Z,pl,2.565559e+09,everyone,291712315,1479126786923548672,"[{'type': 'replied_to', 'id': '147883390151047...","{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,None
939,Twitter Web App,1478035206904950788,@PaczkomatyPL 2/2 do tego reklamacja przez str...,2022-01-06T13:50:39.000Z,pl,8.590470e+17,everyone,859047002381967361,1479088022226452480,"[{'type': 'replied_to', 'id': '147908719495022...","{'retweet_count': 0, 'reply_count': 2, 'like_c...",NaN,None


#Visualisation
Materials:

https://medium.com/@siddhardhan23/data-visualization-in-python-a90ddb706b23

https://towardsdatascience.com/5-python-gui-frameworks-to-create-desktop-web-and-even-mobile-apps-c25f1bcfb561

